# Datapath Example 2
This notebook gives a very basic example of how to access data. 
It assumes that you understand the concepts presented in the 
example 1 notebook.

In [1]:
# Import deriva modules
from deriva_common import ErmrestCatalog, get_credential

In [2]:
# Connect with the deriva catalog
protocol = 'https'
hostname = 'www.facebase.org'
catalog_number = 1
credential = None
# If you need to authenticate, use Deriva Auth agent and get the credential
# credential = get_credential(hostname)
catalog = ErmrestCatalog(protocol, hostname, catalog_number, credential)

In [3]:
# Get the path builder interface for this catalog
pb = catalog.getPathBuilder()

## DataPaths
The `PathBuilder` object allows you to begin `DataPath`s from the base `Table`s. A `DataPath` begins with a `Table` (or an `TableAlias` to be discussed later) as its "root" from which one can "`link`", "`filter`", and fetch its "`entities`".

### Start a path rooted at a table from the catalog
We will reference a table from the PathBuilder `pb` variable from above. Using the PathBuilder, we will reference the "isa" schema, then the "dataset" table, and from that table start a path.

In [4]:
path = pb.schemas['isa'].tables['dataset'].path

We could have used the more compact dot-notation to start the same path.

In [5]:
path = pb.isa.dataset.path

### Getting the URI of the current path
All DataPaths have URIs for the referenced resources in ERMrest. The URI identifies the resources which are available through "RESTful" Web protocols supported by ERMrest.

In [6]:
print(path.uri)

https://www.facebase.org/ermrest/catalog/1/entity/dataset:=isa:dataset


## EntitySets
The entities from a DataPath are accessed through a pythonic container object, the `EntitySet`. The `EntitySet` is returned by the DataPath's `entities()` method.

In [7]:
entities = path.entities()

### Fetch entities from the catalog
Now we can get entities from the server using the EntitySet's `fetch()` method.

In [8]:
entities.fetch()

`EntitySet`s behave like python containers. For example, we can check the count of rows in this EntitySet.

In [9]:
len(entities)

712

**Note**: If we had not explicitly called the `fetch()` method, then it would have been called implicitly on the first container operation such as `len(...)`, `list(...)`, `iter(...)` or get item `[...]`.

### Get an entity
To get one entity from the set, use the usual container operator to get an item.

In [10]:
entities[9]

{'_keywords': 'Research on Functional Genomics, Image Analysis and Rescue of Cleft Palate Released',
 'accession': 'FB00000288.02',
 'description': 'Mouse ID: DH202\n This dataset includes a microCT scan of the skull of a Smad4fl/fl mouse at E18.5.\nThe scan is in NiFTI format. See Help section for a list of free applications to view files in NiFTI format   ',
 'funding': 'This study was supported by grants from the National Institute of Dental and Craniofacial Research, NIH (DE012711, DE014078, DE017007, and DE020065) to Yang Chai.',
 'gene_summary': None,
 'human_anatomic': None,
 'id': 10521,
 'mouse_genetic': None,
 'project': 156,
 'release_date': '2015-06-01',
 'show_in_jbrowse': None,
 'status': 2,
 'study_design': None,
 'summary': 'Mouse ID: DH202; This dataset includes a microCT scan of the skull of a Smad4fl/fl mouse at E18.5. The scan is in NiFTI format, which can be read by a number of free software applications including those listed below. To receive the scan in DICOM fo

### Get a specific attribute value from an entity
To get one attribute value from an entity get the item using its `Column`'s `name` property.

In [11]:
dataset = pb.schemas['isa'].tables['dataset']

print (entities[9][dataset.accession.name])

FB00000288.02


## Fetch a Limited Number of Entities
To set a limit on the number of entities to be fetched from the catalog, use the explicit `fetch(limit=...)` method with the desired upper limit to fetch from the catalog.

In [12]:
entities.fetch(limit=3)
len(entities)

3

### Iterate over the EntitySet
Entitysets are iterable like a typical container.

In [13]:
for entity in entities:
    print(entity[dataset.accession.name])

FB00000827
FB00000861
FB00000380.01


## Convert to Pandas DataFrame
EntitySets can be transformed into the popular Pandas DataFrame.

In [14]:
entities.dataframe

,_keywords,accession,description,funding,gene_summary,human_anatomic,id,mouse_genetic,project,release_date,show_in_jbrowse,status,study_design,summary,thumbnail,title,view_gene_summary,view_related_datasets
0,Epigenetic landscapes and regulatory divergenc...,FB00000827,Tg(hg19_chr7:145843942-145844366::LacZ) activi...,U01 DE024430,None,None,14062,None,307,2016-01-01,None,3,None,Activity of human neural crest enhancer in E11...,442,Activity of human neural crest enhancer in E11...,None,None
1,Developing 3D Craniofacial Morphometry Data an...,FB00000861,Human-subject dataset with scans (in .ply form...,None,None,None,14096,None,301,2017-03-03,None,3,None,Human-subject dataset with scans for the 3D cr...,442,Developing 3D Craniofacial Morphometry Data an...,None,None
2,Functional Analysis of Neural Crest and Palate...,FB00000380.01,microMRI images of skulls of Ctgftm1Kml/Ctgft...,PIs: Scott Fraser and Seth Ruffins. This work ...,None,None,6399,None,151,2015-06-01,None,2,None,microMRI images of skulls of Ctgftm1Kml/Ctgft...,382,microMRI images of skulls of Ctgftm1Kml/Ctgftm...,None,None


## Fetch a Subset of Attributes
It is also possible to fetch only a subset of attributes from the catalog. The `entities(...)` method accepts a variable argument list followed by keyword arguments. Each argument must be a `Column` object from the table's `columns` container. **Remember**, keyword arguments _must come after_ other arguments.

In [15]:
entities = path.entities(dataset.accession, dataset.title, The_Statues_Code=dataset.status).fetch(limit=5)

### Convert to list
Now we can look at the results from the above fetch. To demonstrate a different access mode, we can convert the entities to a standard python list and dump to the console.

In [16]:
list(entities)

[{'The_Statues_Code': 3,
  'accession': 'FB00000827',
  'title': 'Activity of human neural crest enhancer in E11.5 mouse embryo - Tg(hg19_chr7:145843942-145844366::LacZ)'},
 {'The_Statues_Code': 3,
  'accession': 'FB00000861',
  'title': 'Developing 3D Craniofacial Morphometry Data and Tools to Transform Dysmorphology'},
 {'The_Statues_Code': 2,
  'accession': 'FB00000380.01',
  'title': 'microMRI images of skulls of Ctgftm1Kml/Ctgftm1Kml mice at E14.5 '},
 {'The_Statues_Code': 2,
  'accession': 'FB00000393.01',
  'title': 'microMRI images of skulls of Tgfbr2fl/fl mice at E18.5'},
 {'The_Statues_Code': 2,
  'accession': 'FB00000155',
  'title': 'Micro-CT images of adult mouse skulls, Collaborative Cross NZO x WSB'}]

# Implicit DataPath Creation
**Proceed with caution**

For compactness, `Table` objects (and `TableAlias` objects to be discussed in a later example notebook) provide `DataPath`-like methods. E.g., `link(...)`, `filter(...)`, and `entities(...)`, which will implicitly create `DataPath`s rooted at the table and return the newly created path.

### Tables
Begin by getting a base table, "`dataset`", from the "`isa`" schema accessible from the path builder.

**Important**: A `Table` object is **not** a `DataPath` although we provide access methods that make it simple to directly get its entities. We will revisit this distinction later in this notebook.

In [17]:
dataset = pb.isa.dataset